In [ ]:
# # TODO:
# 1) load single trials
# 2) load beh data 
# 3) extract meta data from filenames

In [45]:
import os, glob, re
import nilearn
import pandas as pd
import numpy as np

In [40]:
singletrial_dir = '/Volumes/spacetop_projects_cue/analysis/fmri/nilearn/singletrial'
npy_dir = '/Volumes/spacetop_projects_cue/analysis/fmri/nilearn/deriv05_singletrialnpy'

In [54]:
sub = 'sub-0061'
ses = '*'; run = '*'
task = 'pain'
nii_list = sorted(glob.glob(os.path.join(npy_dir, sub, f"{sub}_{ses}_{run}_runtype-{task}*.npy")))

In [55]:
# extract metadata from filenames
keyword_names = ["sub", "ses", "run", "runtype", "event", "trial", "cuetype", "stimintensity"] # Define the desired keyword names
dfs = [
    pd.DataFrame([dict(zip(keyword_names, re.findall(r'-(.*?)(?:_|$)', os.path.splitext(os.path.basename(nii_fname))[0])))])
    for nii_fname in nii_list
]
metadf = pd.concat(dfs, ignore_index=True)

# Strip leading zeros from specific columns
columns_to_strip = ['sub', 'ses', 'run', 'trial']
metadf[columns_to_strip] = metadf[columns_to_strip].apply(lambda x: x.str.lstrip('0'))

In [56]:
metadf

,sub,ses,run,runtype,event,trial,cuetype,stimintensity
0,61,1,1,pain,stimulus,,low,low
1,61,1,1,pain,stimulus,1,low,med
2,61,1,1,pain,stimulus,2,high,med
3,61,1,1,pain,stimulus,3,low,high
4,61,1,1,pain,stimulus,4,high,med
...,...,...,...,...,...,...,...,...
67,61,4,6,pain,stimulus,7,high,low
68,61,4,6,pain,stimulus,8,low,low
69,61,4,6,pain,stimulus,9,high,med
70,61,4,6,pain,stimulus,10,low,med


In [57]:
beta_array = []
subwise_beta = []
for nii_fname in nii_list:
    beta_array.append(np.load(nii_list[0]).ravel())

subwise_beta = np.vstack(beta_array)

In [58]:
subwise_beta.shape

(72, 458294)

## behavioral data

In [61]:
beh_dir = '/Volumes/spacetop_projects_cue/data/beh/beh03_bids/'
beh_list = sorted(glob.glob(os.path.join(beh_dir, sub, "**", f"*.tsv" ), recursive= True))

In [63]:
# Initialize an empty list to store the DataFrames
beh_dfs = []

# Load and append DataFrames in a loop
for beh_fname in beh_list:
    beh_df = pd.read_csv(beh_fname)
    beh_dfs.append(beh_df)

# Concatenate all DataFrames vertically
combined_df = pd.concat(beh_dfs, ignore_index=True)

In [64]:
combined_df

,Unnamed: 0,onset,duration,trial_type,sub,ses,run,runtype,eventtype,trialnum,cuetype,stimtype,expectrating,outcomerating,singletrial_fname
0,0,14.19410,1.000000,cue,sub-0061,ses-01,run-01,pain,cue,0,low_cue,low_stim,NaN,NaN,sub-0061_ses-01_run-01_runtype-pain_event-cue_...
1,1,43.80500,1.000000,cue,sub-0061,ses-01,run-01,pain,cue,1,low_cue,med_stim,NaN,NaN,sub-0061_ses-01_run-01_runtype-pain_event-cue_...
2,2,77.80085,1.000000,cue,sub-0061,ses-01,run-01,pain,cue,2,high_cue,med_stim,NaN,NaN,sub-0061_ses-01_run-01_runtype-pain_event-cue_...
3,3,110.49621,1.000000,cue,sub-0061,ses-01,run-01,pain,cue,3,low_cue,high_stim,11.900590,NaN,sub-0061_ses-01_run-01_runtype-pain_event-cue_...
4,4,150.71101,1.000000,cue,sub-0061,ses-01,run-01,pain,cue,4,high_cue,med_stim,NaN,NaN,sub-0061_ses-01_run-01_runtype-pain_event-cue_...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
859,43,255.69725,1.467486,outcomerating,sub-0061,ses-04,run-06,pain,outcomerating,7,high_cue,low_stim,126.730038,77.905243,sub-0061_ses-04_run-06_runtype-pain_event-outc...
860,44,288.99300,3.167975,outcomerating,sub-0061,ses-04,run-06,pain,outcomerating,8,low_cue,low_stim,28.228776,62.198541,sub-0061_ses-04_run-06_runtype-pain_event-outc...
861,45,317.20352,2.034380,outcomerating,sub-0061,ses-04,run-06,pain,outcomerating,9,high_cue,med_stim,69.524843,90.230102,sub-0061_ses-04_run-06_runtype-pain_event-outc...
862,46,361.50337,2.206804,outcomerating,sub-0061,ses-04,run-06,pain,outcomerating,10,low_cue,med_stim,55.216880,116.048887,sub-0061_ses-04_run-06_runtype-pain_event-outc...
